In [2]:
import ibm_db,getpass

In [119]:
def sub_group(row, flag=0, mat = None):
    sourceDB = row[0]
    targetDB = row[1]
    s_schema = row[2]
    s_table  = row[3]
    t_schema = row[4]
    t_table  = row[5]
    records  = row[6]           
    filters  = row[7]


    subname = env + '_' + targetDB + '_' + sourceDB + '_' + t_schema.strip('0123456789_')
    sourceid = env[:1] +  sourceDB[:1] + targetDB[:1] + '_' + t_schema.strip('0123456789_')[:4]
    if (flag == 0 ):
        pass
    else:  ## If create seperate subs for big table, renew the sourceID and subname with number
        subname = subname + str(flag) + '_' + "".join(e[0] for e in t_table.split('_'))
        sourceid = sourceid[:8 - len(str(flag))] + str(flag)


    if ( mat is None or not re.match(t_schema.strip('0123456789_'),mat)):
            mat = t_schema.strip('0123456789_')
            print ( "\n\n"*2, "#"* 170,"\n"*2)
            print ( "# subscription for schema ", mat,"\n\n")
            print ( "add subscription name {} sourceid {};". format(subname.upper(), sourceid.upper()))
            print ( "select subscription name {};". format(subname.upper()))
            
            print ( "lock subscription name {};". format(subname.upper()))
    elif(mat is not None and re.match(t_schema.strip('0123456789_'),mat)):
        # If next table schema is same as the previous one, generate table mapping directly
        pass
    print("add table mapping sourceSchema {0:<12} sourceTable {1:<42} targetSchema {2:<12} targetTable {3:<42} method refresh;".format(s_schema,s_table,t_schema,t_table), "# Table filters: ", filters, "Record:", records)
    return mat

In [4]:
print("Please check db connections before run this script.\n", "*"*50 )
username = input("Database Username:")
password = getpass.getpass("Database Password:")

Please check db connections before run this script.
 **************************************************


In [123]:
connstr = r'DATABASE=ODS;HOSTNAME=commander.rtp.dst.ibm.com;PORT=62010;PROTOCOL=TCPIP;UID=' + username + ';PWD=' + password + r';SECURITY=SSL;SSLCLIENTKEYSTOREDB=/Users/arthur/Documents/Work/DBA/DB2/ssl/mydbclient.kdb;SSLCLIENTKEYSTASH=/Users/arthur/Documents/Work/DBA/DB2/ssl/mydbclient.sth;'
conn = ibm_db.connect(connstr, "", "")

In [124]:
import re

In [125]:
sql = "SELECT 'ODS' SOURCE_SERVER, 'CSDW' TARGET_SERVER, trim(tabschema) SOURCE_SCHEMA, trim(tabname) SOURCE_TABLE,  trim(tabschema) TARGET_SCHEMA, trim(tabname) TARGET_TABLE, CARD RECORDS, '' FILTERS\
    FROM syscat.tables \
    WHERE TYPE =  'T' \
    AND tabschema NOT LIKE 'SYS%' \
    AND tabschema NOT LIKE 'DB2%' \
    AND TABSCHEMA  NOT IN ('ASN','ASN_EXT','ASNPAN','ASN_MTH','CDC','UTOL','REPL','CLEVEILL') \
    ORDER BY TABSCHEMA,card,TABNAME  \
    "

# sql = "SELECT trim(tabschema) SOURCE_SCHEMA, trim(tabname) SOURCE_TABLE,  trim(tabschema) TARGET_SCHEMA, trim(tabname) TARGET_TABLE\
#         FROM syscat.tables \
#         WHERE TYPE = 'T'\
#         AND (tabschema LIKE 'CEDP%' \
#         or tabschema LIKE 'DSP%' \
#         or tabschema LIKE 'BRIOADMN%' \
#         or tabschema LIKE 'AUDIT%' )\
#             ORDER BY TABSCHEMA,TABNAME "


# sql = "SELECT SOURCE_SERVER, TARGET_SERVER, m.SOURCE_OWNER SOURCE_SCHEMA, m.SOURCE_TABLE SOURCE_TABLE, m.TARGET_OWNER TARGET_SCHEMA, m.TARGET_TABLE TARGET_TABLE, m.PREDICATES FILTERS \
#         FROM \
#         ASN.IBMSNAP_SUBS_SET SET, \
#         ASN.IBMSNAP_SUBS_MEMBR M \
#         WHERE SET.apply_qual = M.apply_qual \
#         AND SET.set_name = M.set_name \
#         AND SOURCE_SERVER IN ('ODS', 'DSS', 'CSDW', 'WEB')"


In [126]:

result = ibm_db.exec_immediate(conn, sql)
# sourceDB = input('Please input sourceDB:')
# targetDB = input('Please input targetDB:')
env = input('Please input Env (VT/QA/PROD):')

mat = None
count = set()
## SQL statement contain at least 7 columns: SOURCE_DB, TARGET_DB, SOURCE_SCHEMA, SOURCE_TABLE, TARGET_SCHEMA, TARGET_TABLE, FILTERS
print("Connect to access and datastore","\n" , "#"*150)
print("connect server hostname %hostname% port %port% username %username% password %password%;")
print("connect datastore name %source% context source;")
print("connect datastore name %target% context target;")
row = ibm_db.fetch_tuple(result)
seperate_sub = []
while( row ):
     # ##  Create seperate subscription when table recored is greater than 20 million
    if (row[6] <= 20000000):
        
        ## Group table mappings by target schema, even they are in different stage
       mat = sub_group(row,0,mat) 
       count.add(mat)
    else:
        seperate_sub.append(row)
    
    row = ibm_db.fetch_tuple(result)

# Create seperate sub when records more than 20 million
mat = None
c = 1
for row in seperate_sub:

    if (row[6] <= 50000000):
        if(mat_hat != mat):
            c = 1
        else:
            c += 1
            # mat = None

        mat_hat = mat
        mat = sub_group(row, c, mat)
        count.add(mat+str(c))
    else:
        mat = None
        c += 1
        mat = sub_group(row, c, mat)
        count.add(mat+str(c))
        
        mat = None
     
     


print(sorted(count))      
    
    # if (row[4].strip('0123456789') in count):
    #     mat = sub_group(row, c, mat)
    # else:
    #     mat = sub_group(row, c, mat)


## After all table mapped, disconnect access server
print("\n"*2,"#"* 170,"\n # Total subscriptions:", len(count))
print("\ndisconnect server;")


Connect to access and datastore 
 ######################################################################################################################################################
connect server hostname %hostname% port %port% username %username% password %password%;
connect datastore name %source% context source;
connect datastore name %target% context target;




 ########################################################################################################################################################################## 


# subscription for schema  AUDIT 


add subscription name TEMP_CSDW_ODS_AUDIT sourceid TOC_AUDI;
select subscription name TEMP_CSDW_ODS_AUDIT;
lock subscription name TEMP_CSDW_ODS_AUDIT;
add table mapping sourceSchema AUDIT        sourceTable TRIGGER_LIST                               targetSchema AUDIT        targetTable TRIGGER_LIST                               method refresh; # Table filters:   Record: 15
add table mapping sourceSchema AUDIT   

In [ ]:
int('1000')

In [ ]:
# if ibm_db.active(conn):
#     sql = "SELECT DISTINCT TABSCHEMA \
#     FROM syscat.tables \
#     WHERE TYPE = 'T'\
#     AND tabschema NOT LIKE 'SYS%'\
#     AND tabschema NOT LIKE 'DB2%'\
#     AND TABSCHEMA  NOT IN ('ASN','ASN_EXT','ASNPAN','ASN_MTH','CDC','¬','REPL','CLEVEILL') WITH ur"
#     stmt = ibm_db.exec_immediate(conn, sql)
#     while( ibm_db.fetch_row(stmt) ):
#         name = ibm_db.result(stmt, "TABSCHEMA")
#         print("{}".format( name))
#         # print (result['TABNAME'])
# else:
#     print(ibm_db.conn_errormsg())

In [ ]:
ibm_db.close(conn)